# 07-2. 심층 신경망
> p. 367

In [1]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target)
# 60000개의 데이터를 학습용 세트와 검증용 세트로 나눠주기

In [3]:
# 인공 신경망 모델의 층을 만들어주자.
# 이번에는 그 층을 2개 추가해보겠다.

# 입력층과 출력층 사이에 밀집층을 추가하겠다.
# `입력층` - `은닉층` - `출력층`
# 즉 은닉층을 넣겠다.

In [ ]:
# p.369
# 은닉층에도 활성화 함수를 적용해서, 0~1사이의 값으로 계산이 될 수 있도록 설정하겠다.

#### 심층 신경망 만들기
> dense 1 과 dense2 객체를 sequential 클래스에 추가하여 심층 신경망을 만들기

In [15]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
# 확률이 중요한게 아니라 0~1의 값이 필요하므로 위의 코드는 시그모이드 함수를
dense2 = keras.layers.Dense(10, activation='softmax')
# 10개의 레이어는 소프트 맥스 함수를

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# 모델을 만들자
model = keras.Sequential([dense1, dense2])

In [17]:
model.summary()
# p.372

# 784의 데이터에 100개의 뉴런 + 100개의 은닉층 뉴런개수

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# model = keras.Sequential()
# model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
# model.add(dense2)
# add()함수로 dense 를 추가
# model = keras.Sequential([dense1, dense2]) 이것을 이렇게도 쓸 수 있다.

In [20]:
# 학습시키기 이전에 compile 과정을 거치자.
# 이 모델을 어떻게 학습시킬지 정해주는 절차

model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# accuracy 적을 때, 리스트 형식으로 [] 넣기
model.fit(train_scaled, train_target, epochs=5)

# accuracy가 아까 은닉층을 하나만 적용했을 때보다 그 값이 조금 더 높아진 것을 볼 수 있다.

Epoch 1/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8822 - loss: 0.3242
Epoch 2/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8840 - loss: 0.3149
Epoch 3/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8874 - loss: 0.3079
Epoch 4/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8909 - loss: 0.2954
Epoch 5/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8944 - loss: 0.2861


In [21]:
model.evaluate(val_scaled, val_target)

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8821 - loss: 0.3304


[0.32269883155822754, 0.883733332157135]

#### 렐루 함수
> p. 377
- 아주 작은 확률 값 (like 0.1112211) 은 계산하는데 의미가 없다. (의미가 없는 수치이다.) => 버리자!
- 다 0으로 인식하자. / 음수는 쓰지 말자.

`평탄화` : `Flatten`  클래스 - 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할

In [22]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
# 우리 사진 데이터의 크기를 튜플로 알려주자.
#(28, 28)
model.add(keras.layers.Dense(100, activation='relu'))
# sigmoid모다 빠른 함수, relu함수를 적용해보자.
model.add(keras.layers.Dense(10, activation='softmax'))

#  새로운 모델이 생성되었다.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [23]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# 데이터 불러오는 코드를 한번 더 씀
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.
# flatten (평탄화)를 해줬기 때문에, reshape 과정을 생략해도 된다. => train_scaled = train_scaled.reshape(-1, 28*28)
# flatten 은 편의성을 위해 사용한다.
train_scaled, cal_scaled, train_target, val_target = train_test_split(train_scaled, train_target)

In [26]:
# 학습을 시켜주자.
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)
# fit으로 학습용 데이터를 넣어주자

Epoch 1/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7604 - loss: 0.6868
Epoch 2/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8526 - loss: 0.4137
Epoch 3/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8673 - loss: 0.3623
Epoch 4/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8760 - loss: 0.3428
Epoch 5/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8821 - loss: 0.3267


In [ ]:
# 이 과정에서의 결론은 렐루함수 사용이 시그모이드 함수를 쓸 때와 별로 차이가 없더라 (성능에는 별 차이가 없더)
# 컴퓨터 입장에서는 렐루함수가 계산시간을 단축시켜준다.

#### 이제 모델의 성능을 더 올려보자.
#### 옵티마이저 (p. 380)
> 최적화를 시키자.
- 수치들을 지정하는 공간을 조작하자.

```
- SGD 클래스
  - 기본 경사 하강법 옵티마이저
  - 적응적 학습률 옵티마이저
```

In [28]:
model.compile(
    optimizer='adam',
    # adam이라는 옵티마이저를 사용해보자.
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8873 - loss: 0.3082
Epoch 2/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8952 - loss: 0.2886
Epoch 3/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9001 - loss: 0.2667
Epoch 4/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9033 - loss: 0.2624
Epoch 5/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9052 - loss: 0.2535


In [ ]:
# adam은 처음부터 빠른 속도로 경사를 내려간다.
# 그래서 epoch 1부터 높은 accuracy를 보인다.

# 각 optimizer의 특성은 공식 문서를 통해 확인해볼 수 있다.